# 百度AI——驾驶行为分析
针对车载场景，识别驾驶员使用手机、抽烟、不系安全带、双手离开方向盘等动作姿态，分析预警危险驾驶行为，提升行车安全性。

# 开始我们的实验
(1)登录百度AI开发平台，选择“人体分析”，使用“创建应用”建立一个驾驶行为分析应用。 获取应用的AppID、API Key、Secret Key，注意:Secret Key要先选择显示，然后复制其现实的字符。

例如我创建的驾驶行为分析应用获取的AppID、API Key、Secret Key如下：

    """ 公开课的限定量人体行为分析，需要更改为自己的api接口 """
    pic_APP_ID = '21258767'
    pic_API_KEY = 'epRpthFLqIW7PHNWfYP9NqK5'
    pic_SECRET_KEY = 'RF2yO8OIfvcrI4WlC3w0In0G6Upo7GFK'

    
百度AI开发平台注册链接:https://login.bce.baidu.com/?account=&redirect=http%3A%2F%2Fconsole.bce.baidu.com%2F%3Ffromai%3D1#/aip/overview

(2)熟悉我封装的百度A驾驶行为分析的模块 在代码中输入如下代码，导入我的封装的模块

    import sys
    sys.path.append('../baidu_api_lib')
    from baidu_picture import baidu_picture_2_msg
    封装库提供两个方法：初始化方法，驾驶行为分析方法。

<1>初始化方法，需要传入三个参数，就是上步我们创建“驾驶行为分析”应用获取的三个参数 参数信息如下：

    """ 公开课的限定量人体行为分析，需要更改为自己的api接口 """
    pic_APP_ID = '21258767'
    pic_API_KEY = 'epRpthFLqIW7PHNWfYP9NqK5'
    pic_SECRET_KEY = 'RF2yO8OIfvcrI4WlC3w0In0G6Upo7GFK'

调用方式：

    # 传入百度AI的参数，进行图像识别
    pic_msg = baidu_picture_2_msg(pic_APP_ID, pic_API_KEY, pic_SECRET_KEY)
    初始化完毕后，获取pic_msg对象，操作该对象就能进行动物识别。
<2>驾驶行为分析方法 这是我封装百度驾驶行为分析方法的函数原型：

    # 传入百度AI的参数，进行图像识别
    pic_msg = baidu_picture_2_msg(pic_APP_ID, pic_API_KEY, pic_SECRET_KEY)
    
    baidu_request_url:百度AI的调用接口，
    #百度AI的调用url
    baidu_request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/animal"

    'read_word.jpg':进行驾驶行为分析的图片。
    返回值:response为文字识别的信息

    response.json()["person_num"]:检测到的总人数（包括驾驶员和乘客），0代表未监测到驾驶员
    response.json()["driver_num"]:检测到的驾驶员数目。
    response.json()["person_info"]:驾驶员的属性行为信息；若未检测到驾驶员，则该项为[]
    response.json()["person_info"][0][attributes]:第一个驾驶员的所有行为分析结果，包含是否吸烟、是否使用手机、是否未系安全带等。

    ...还有不少参数，通过print(response.json())可以发现

In [ ]:
######################################################
#
# 先感受下小派驾驶行为分析的本领,选择本cell，按shirt+enter键运行本模块
#
######################################################

#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Copyright (c) 2020 Taste all Pi.
#
# Licensed under the GNU General Public License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.gnu.org/licenses/gpl-2.0.html
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#导入标准库
import sys
import os
from playsound import playsound
import cv2 as cv
import time
from multiprocessing import Process, Queue
import multiprocessing

#导入自定义库
sys.path.append('../baidu_api_lib')
from baidu_picture import baidu_picture_2_msg
from baidu_sound import baidu_word_2_sound

""" 公开课的限定量人体行为分析，需要更改为自己的api接口 """
pic_APP_ID = '21258767'
pic_API_KEY = 'epRpthFLqIW7PHNWfYP9NqK5'
pic_SECRET_KEY = 'RF2yO8OIfvcrI4WlC3w0In0G6Upo7GFK'

#百度AI的调用url
baidu_request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/driver_behavior"

#　获取摄像头图像
def camera_frame_func(task_name, mult_queue1, mydict):
    
    # 创建一个VideoCapture对象
    capture = cv.VideoCapture(0) 
    
    #更新人脸识别信息
    mydict["get_pic"] = "false"
    
    #　给出提示信息
    print(task_name + "任务启动")
    
    try:
        while True:
            # 一帧一帧读取视频
            ret, frame = capture.read()
            
            #将拍摄到的图片发送到消息队列中
            if mydict["get_pic"] == "true":
                mydict["get_pic"] = "false"
                mult_queue1.put(frame)

            # 本地显示视频图像
            cv.imshow('real_time picture', frame) 
            cv.waitKey(1)

    except KeyboardInterrupt:
        # 释放cap,销毁窗口
        capture.release()    
        #关闭显示窗口
        cv.destroyAllWindows() 
        print(task_name + "任务被终止")
        
#处理图像
def proc_frame_func(task_name, mult_queue, mydict):
    #　给出提示信息
    print(task_name + "任务启动")
    
    # 传入百度AI的参数，进行图像识别
    pic_msg = baidu_picture_2_msg(pic_APP_ID, pic_API_KEY, pic_SECRET_KEY)
    
    #传入百度AI的参数，进行语音合成
    word_2_sound = baidu_word_2_sound(pic_APP_ID, pic_API_KEY, pic_SECRET_KEY)
    word_2_sound.trans_word_to_sound("欢迎使用小派安全驾驶辅助系统",'tst_sound.mp3')
    os.system('mplayer ' + 'tst_sound.mp3')
    time.sleep(2)
    
    try:
       
        while True:
            #连续获取图片，取出第三章
            loop_num = 0
            while loop_num < 3:
                #从队列中获取图片，显示图像质量
                mydict["get_pic"] = "true"
                frame = mult_queue.get()
                cv.waitKey(1)
                loop_num += 1
                time.sleep(1)
            
            #关闭显示窗口
            cv.destroyAllWindows()            

            # 写入图片
            cv.imwrite('read_word.jpg',frame)
            
            #从百度AI获取图片分析结果
            response = pic_msg.pic_2_msg(baidu_request_url, 'read_word.jpg')
            
            #给出百度AI分析的数据
            print(response.json())
            
            #识别出驾驶员了，就要根据识别信息给出提示
            if response.json()["driver_num"] > 0:
                #获取驾驶员的驾驶状态
                driver_attr = response.json()['person_info'][0]['attributes']
                
                #未佩戴安全带概率大于0.5（概率大于50%）
                if driver_attr['not_buckling_up']['score'] > 0.5:
                    word_2_sound.trans_word_to_sound("安全驾驶，请先系好安全带",'tst_sound.mp3')
                    os.system('mplayer ' + 'tst_sound.mp3')
                    
                #未带口罩概率大于0.5（概率大于50%）
                if driver_attr['no_face_mask']['score'] > 0.5:
                    word_2_sound.trans_word_to_sound("请带好口罩，为抗疫胜利尽份力",'tst_sound.mp3')
                    os.system('mplayer ' + 'tst_sound.mp3')
                     
                #打电话概率大于0.5（概率大于50%）
                if driver_attr['cellphone']['score'] > 0.5:
                    word_2_sound.trans_word_to_sound("安全驾驶，请不要拨打电话",'tst_sound.mp3')
                    os.system('mplayer ' + 'tst_sound.mp3')
                    
                #双手远离方向盘概率大于0.5（概率大于50%）
                elif driver_attr['both_hands_leaving_wheel']['score'] > 0.5:
                    word_2_sound.trans_word_to_sound("安全驾驶，请握紧方向盘",'tst_sound.mp3')
                    os.system('mplayer ' + 'tst_sound.mp3')    
                       
                #打哈欠概率大于0.5（概率大于50%）
                elif driver_attr['yawning']['score'] > 0.5:
                    word_2_sound.trans_word_to_sound("您有点困了，请停车休息",'tst_sound.mp3')
                    os.system('mplayer ' + 'tst_sound.mp3')      
                    
                #低头概率大于0.5（概率大于50%）
                elif driver_attr['head_lowered']['score'] > 0.5:
                    word_2_sound.trans_word_to_sound("安全驾驶，请注视前方",'tst_sound.mp3')
                    os.system('mplayer ' + 'tst_sound.mp3')      
                    
                #吸烟概率大于0.5（概率大于50%）
                elif driver_attr['smoke']['score'] > 0.5:
                    word_2_sound.trans_word_to_sound("为了您和他人的健康，请不要在车内吸烟",'tst_sound.mp3')
                    os.system('mplayer ' + 'tst_sound.mp3')
                
                #其他情况，可以自行添加
                else:
                    pass
            
            #等待10秒后，在进行图像采集
            time.sleep(10)
            
    except KeyboardInterrupt:
        os.remove('tst_sound.mp3')
        os.remove('read_word.jpg')
        print(task_name + "任务被终止")
        
if __name__ == "__main__":
    try:
        
        mydict=multiprocessing.Manager().dict()
        
        #　定义传递图像队列和传递图像处理结果队列
        q_frame = Queue()
        
        #　采集摄像头进程、处理图片进程、播报语音信息
        get_camera_frame = Process(target=camera_frame_func, args=("获取摄像头图像", q_frame, mydict))
        proc_frame       = Process(target=proc_frame_func, args=("处理图像", q_frame, mydict))
        
        # 启动任务
        get_camera_frame.start()
        proc_frame.start()

    except KeyboardInterrupt:
        print("任务被终止了")

# 让我们临摹代码，学习“驾驶行为分析”功能

In [ ]:
# #导入标准库
# import sys

# import os

# from playsound import playsound

# import cv2 as cv

# import time

# from multiprocessing import Process, Queue

# import multiprocessing


# #导入自定义库
# sys.path.append('../baidu_api_lib')

# from baidu_picture import baidu_picture_2_msg

# from baidu_sound import baidu_word_2_sound


# """ 公开课的限定量人体行为分析，需要更改为自己的api接口 """
# pic_APP_ID = '21258767'

# pic_API_KEY = 'epRpthFLqIW7PHNWfYP9NqK5'

# pic_SECRET_KEY = 'RF2yO8OIfvcrI4WlC3w0In0G6Upo7GFK'



# #百度AI的调用url
# baidu_request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/driver_behavior"


# #　获取摄像头图像
# def camera_frame_func(task_name, mult_queue1, mydict):

    
#     # 创建一个VideoCapture对象
#     capture = cv.VideoCapture(0) 

    
#     #更新人脸识别信息
#     mydict["get_pic"] = "false"

    
#     #　给出提示信息
#     print(task_name + "任务启动")

    
#     try:

#         while True:

#             # 一帧一帧读取视频
#             ret, frame = capture.read()

            
#             #将拍摄到的图片发送到消息队列中
#             if mydict["get_pic"] == "true":

#                 mydict["get_pic"] = "false"

#                 mult_queue1.put(frame)


#             # 本地显示视频图像
#             cv.imshow('real_time picture', frame) 

#             cv.waitKey(1)


#     except KeyboardInterrupt:

#         # 释放cap,销毁窗口
#         capture.release()   

#         #关闭显示窗口
#         cv.destroyAllWindows() 

#         print(task_name + "任务被终止")

        
# #处理图像
# def proc_frame_func(task_name, mult_queue, mydict):

#     #　给出提示信息
#     print(task_name + "任务启动")

    
#     # 传入百度AI的参数，进行图像识别
#     pic_msg = baidu_picture_2_msg(pic_APP_ID, pic_API_KEY, pic_SECRET_KEY)

    
#     #传入百度AI的参数，进行语音合成
#     word_2_sound = baidu_word_2_sound(pic_APP_ID, pic_API_KEY, pic_SECRET_KEY)

#     word_2_sound.trans_word_to_sound("欢迎使用小派安全驾驶辅助系统",'tst_sound.mp3')

#     os.system('mplayer ' + 'tst_sound.mp3')

#     time.sleep(2)

    
#     try:

       
#         while True:

#             #连续获取图片，取出第三章
#             loop_num = 0

#             while loop_num < 3:

#                 #从队列中获取图片，显示图像质量
#                 mydict["get_pic"] = "true"

#                 frame = mult_queue.get()

#                 cv.imshow('I get the picture', frame) 

#                 cv.waitKey(1)

#                 loop_num += 1

#                 time.sleep(1)

            
#             #关闭显示窗口
#             cv.destroyAllWindows()    


#             # 写入图片
#             cv.imwrite('read_word.jpg',frame)

            
#             #从百度AI获取图片分析结果
#             response = pic_msg.pic_2_msg(baidu_request_url, 'read_word.jpg')

            
#             #给出百度AI分析的数据
#             #print(response.json())

            
#             #识别出驾驶员了，就要根据识别信息给出提示
#             if response.json()["driver_num"] > 0:

#                 #获取驾驶员的驾驶状态
#                 driver_attr = response.json()['person_info'][0]['attributes']

                
#                 #未佩戴安全带概率大于0.5（概率大于50%）
#                 if driver_attr['not_buckling_up']['score'] > 0.5:

#                     word_2_sound.trans_word_to_sound("安全驾驶，请先系好安全带",'tst_sound.mp3')

#                     os.system('mplayer ' + 'tst_sound.mp3')

                    
#                 #未带口罩概率大于0.5（概率大于50%）
#                 if driver_attr['no_face_mask']['score'] > 0.5:

#                     word_2_sound.trans_word_to_sound("请带好口罩，为抗疫胜利尽份力",'tst_sound.mp3')

#                     os.system('mplayer ' + 'tst_sound.mp3')

                     
#                 #打电话概率大于0.5（概率大于50%）
#                 if driver_attr['cellphone']['score'] > 0.5:

#                     word_2_sound.trans_word_to_sound("安全驾驶，请不要拨打电话",'tst_sound.mp3')

#                     os.system('mplayer ' + 'tst_sound.mp3')

                    
#                 #双手远离方向盘概率大于0.5（概率大于50%）
#                 elif driver_attr['both_hands_leaving_wheel']['score'] > 0.5:

#                     word_2_sound.trans_word_to_sound("安全驾驶，请握紧方向盘",'tst_sound.mp3')

#                     os.system('mplayer ' + 'tst_sound.mp3')    

                       
#                 #打哈欠概率大于0.5（概率大于50%）
#                 elif driver_attr['yawning']['score'] > 0.5:

#                     word_2_sound.trans_word_to_sound("您有点困了，请停车休息",'tst_sound.mp3')

#                     os.system('mplayer ' + 'tst_sound.mp3')      

                    
#                 #低头概率大于0.5（概率大于50%）
#                 elif driver_attr['head_lowered']['score'] > 0.5:

#                     word_2_sound.trans_word_to_sound("安全驾驶，请注视前方",'tst_sound.mp3')

#                     os.system('mplayer ' + 'tst_sound.mp3')      

                    
#                 #吸烟概率大于0.5（概率大于50%）
#                 elif driver_attr['smoke']['score'] > 0.5:

#                     word_2_sound.trans_word_to_sound("为了您和他人的健康，请不要在车内吸烟",'tst_sound.mp3')

#                     os.system('mplayer ' + 'tst_sound.mp3')

                
#                 #其他情况，可以自行添加
#                 else:

#                     pass

            
#             #等待10秒后，在进行图像采集
#             time.sleep(10)

            
#     except KeyboardInterrupt:

#         os.remove('tst_sound.mp3')

#         os.remove('read_word.jpg')

#         print(task_name + "任务被终止")
        
# if __name__ == "__main__":

#     try:

        
#         mydict=multiprocessing.Manager().dict()

        
#         #　定义传递图像队列和传递图像处理结果队列
#         q_frame = Queue()

        
#         #　采集摄像头进程、处理图片进程、播报语音信息
#         get_camera_frame = Process(target=camera_frame_func, args=("获取摄像头图像", q_frame, mydict))

#         proc_frame       = Process(target=proc_frame_func, args=("处理图像", q_frame, mydict))

        
#         # 启动任务
#         get_camera_frame.start()

#         proc_frame.start()


#     except KeyboardInterrupt:

#         print("任务被终止了")


# 课后练习

1、我们继续完善小派的安全驾驶，我们可以添加：发现驾驶员没有系安全带、未带口罩等不安全驾驶行为时，将照片保存下来，并在照片上记录上时间。

2、我们还可以增加行车记录功能，将驾驶全过程进行视频记录。